In [72]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

class KiteEnv(Env):
    def __init__(self):
        self.FlyLength = 60
        self.pushPullSpeed = 5
        self.maxWindSpeed = 40
        self.minWindSpeed = 10
        self.observation_space = Box(low=np.array([10.]), high=np.array([40.]), dtype=np.float32)
        self.action_space = Discrete(3)
        self.windspeed = 30 + random.randint(-3,3)
        self.hight = 10 + random.randint(-2,2)

    def step(self, action):
        self.FlyLength -= 1
        # -1 = push. 0 = hold, 1 = release
        effective_speed = self.windspeed + action*5

        if self.FlyLength <= 0:
            done = True
        else:
            done = False
        self.windspeed += random.randint(-5,5)

        newHight = self.hight

        reward = 0
        if done or effective_speed >= self.maxWindSpeed:
            if action != 1:
                reward = -30
            else:
                reward = 0
            done = True
        else:
            newHight =+ 0.25*(action)

        if not done:
            if effective_speed <= self.minWindSpeed:
                reward += -10
                if effective_speed <= 0:
                    done = True
                    reward += -20
            else:
                reward += (newHight - self.hight)*4
        self.hight = newHight

        info = {}

        return self.windspeed, reward, done, info

    def render(self, mode='human'):
        pass

    def reset(self):
        self.windspeed = 30 + random.randint(-3,3)
        self.hight = 10 + random.randint(-2,2)
        self.FlyLength = 60
        return self.windspeed


In [73]:
env = KiteEnv()
env.observation_space.sample()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


array([26.265148], dtype=float32)

In [74]:
ep = 10
for e in range(ep):
  state = env.reset()
  done = False
  score = 0
  while not done:
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:{} Score:{}'.format(e+1, score))

Episode:1 Score:-69.0
Episode:2 Score:-66.0
Episode:3 Score:-54.0
Episode:4 Score:-78.0
Episode:5 Score:-30
Episode:6 Score:-73.0
Episode:7 Score:-70.0
Episode:8 Score:-61.0
Episode:9 Score:-78.0
Episode:10 Score:-36.0


In [75]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [76]:
states = env.observation_space.shape
actions = env.action_space.n

In [77]:
states

(1,)

In [78]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [79]:
model = build_model(states, actions)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 24)                48        
_________________________________________________________________
dense_14 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [80]:
!pip install keras-rl2

In [81]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [82]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [83]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 24:37 - reward: -38.0000

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 85s 9ms/step - reward: -7.2975
1491 episodes - episode_reward: -48.920 [-167.000, -30.000] - loss: 71.151 - mae: 29.696 - mean_q: -29.079

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: -9.4087
2047 episodes - episode_reward: -45.965 [-86.000, -30.000] - loss: 67.576 - mae: 32.268 - mean_q: -38.910

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: -8.6944
1862 episodes - episode_reward: -46.687 [-86.000, -30.000] - loss: 64.763 - mae: 33.627 - mean_q: -39.360

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 95s 10ms/step - reward: -9.2544
2019 episodes - episode_reward: -45.835 [-82.000, -30.000] - loss: 65.312 - mae: 33.752 - mean_q: -39.617

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 95s 9ms/step - reward: -9.1823
done, took 457.594 seconds


In [84]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -30.000, steps: 1
Episode 2: reward: -30.000, steps: 1
Episode 3: reward: -72.000, steps: 15
Episode 4: reward: -68.000, steps: 48
Episode 5: reward: -65.000, steps: 2
Episode 6: reward: -30.000, steps: 1
Episode 7: reward: -30.000, steps: 1
Episode 8: reward: -68.000, steps: 41
Episode 9: reward: -30.000, steps: 1
Episode 10: reward: -30.000, steps: 1
Episode 11: reward: -30.000, steps: 1
Episode 12: reward: -72.000, steps: 2
Episode 13: reward: -77.000, steps: 6
Episode 14: reward: -30.000, steps: 1
Episode 15: reward: -72.000, steps: 2
Episode 16: reward: -72.000, steps: 13
Episode 17: reward: -30.000, steps: 1
Episode 18: reward: -69.000, steps: 19
Episode 19: reward: -30.000, steps: 1
Episode 20: reward: -30.000, steps: 1
Episode 21: reward: -30.000, steps: 1
Episode 22: reward: -30.000, steps: 1
Episode 23: reward: -30.000, steps: 1
Episode 24: reward: -73.000, steps: 2
Episode 25: reward: -72.000, steps: 2
Episode 26: reward: -30.0